# Graphormer笔记（三）
> 代码来自于：https://github.com/ytchx1999/Graphormer/blob/main/graphormer/
本文的代码还是来自上述仓库，该仓库实际上是clone的Graphormer官方仓库的v1版本，现在官方版本升级到v2版本了主要是增加了fairseq框架训练等功能，主要是工程上的改进，核心的内容没有改变，因此我们只分析老版了解原理即可。


### 1、虚拟节点的添加及编码：
```python
class Graphormer(pl.LightningModule):
    def __init__(......):
	......
		self.graph_token = nn.Embedding(1, hidden_dim)
        self.graph_token_virtual_distance = nn.Embedding(1, num_heads)
	......
 	def forward(self, batched_data, perturb=None):
	......
		t = self.graph_token_virtual_distance.weight.view(1, self.num_heads, 1)
        graph_attn_bias[:, :, 1:, 0] = graph_attn_bias[:, :, 1:, 0] + t
        graph_attn_bias[:, :, 0, :] = graph_attn_bias[:, :, 0, :] + t
	......

		graph_token_feature = self.graph_token.weight.unsqueeze(
            0).repeat(n_graph, 1, 1)
        graph_node_feature = torch.cat(
            [graph_token_feature, node_feature], dim=1)
	......
```
可以看到虚拟节点用embedding为1的编码为特征为hidden feature维度的特征，最后与节点特征concat操作即可；虚拟节点pos bias嵌入为输出为head数维度，添加到整个注意力bias的第一行与第一列。
```python
		node_feature = node_feature + \
		self.in_degree_encoder(in_degree) + \
		self.out_degree_encoder(out_degree)  # degree encoder
```
这一部分是入度与出度的编码过程，编码完成后，与上述的虚拟节点特征concat起来，其实这里的虚拟节点类似于nlp里的cls token。

### 2、边特征bias嵌入及编码：
这一步介绍multi hop的嵌入及编码，源代码中也有一段用cython接口写的计算Floyd最短距离矩阵及最短路径特征的代码很有意思，不是这篇论文的重点这里就不做介绍了，只需要知道其输出为最短距离矩阵，edge_input输出的是最短路径上所有经过的边的特征，可能之后有时间再仔细阅读分析。
```python
class Graphormer(pl.LightningModule):
    def __init__(......):
	......
		if self.edge_type == 'multi_hop':
                self.edge_dis_encoder = nn.Embedding(
                    128 * num_heads * num_heads, 1)
		self.edge_encoder = nn.Embedding(
                512 * 3 + 1, num_heads, padding_idx=0)
	......
	def forward(self,...):
	......
        # edge feature
        if self.edge_type == 'multi_hop':
            rel_pos_ = rel_pos.clone()
            rel_pos_[rel_pos_ == 0] = 1  # set pad to 1
            # set 1 to 1, x > 1 to x - 1
            rel_pos_ = torch.where(rel_pos_ > 1, rel_pos_ - 1, rel_pos_)
            if self.multi_hop_max_dist > 0:
                rel_pos_ = rel_pos_.clamp(0, self.multi_hop_max_dist)
                edge_input = edge_input[:, :, :, :self.multi_hop_max_dist, :]
            # [n_graph, n_node, n_node, max_dist, n_head]
            edge_input = self.edge_encoder(edge_input).mean(-2)
            max_dist = edge_input.size(-2)
            edge_input_flat = edge_input.permute(
                3, 0, 1, 2, 4).reshape(max_dist, -1, self.num_heads)
            edge_input_flat = torch.bmm(edge_input_flat, self.edge_dis_encoder.weight.reshape(
                -1, self.num_heads, self.num_heads)[:max_dist, :, :])
            edge_input = edge_input_flat.reshape(
                max_dist, n_graph, n_node, n_node, self.num_heads).permute(1, 2, 3, 0, 4)
            edge_input = (edge_input.sum(-2) /
                          (rel_pos_.float().unsqueeze(-1))).permute(0, 3, 1, 2)
		else:
            # [n_graph, n_node, n_node, n_head] -> [n_graph, n_head, n_node, n_node]
            edge_input = self.edge_encoder(
                attn_edge_type).mean(-2).permute(0, 3, 1, 2)

        graph_attn_bias[:, :, 1:, 1:] = graph_attn_bias[:,
                                                        :, 1:, 1:] + edge_input  # edge encoder
```
我们可以看到边的编码分两种，分别是有multi hop多跳和普通编码，这里边的区别是multi hop是聚合了节点与节点之间最短路径（其中一条路径）的边的属性的融合，普通边属性嵌入则是只将邻接的边的属性添加到对应位置的节点上。
初始化中申请的`edge_dis_encoder`和`edge_encoder`又有什么区别呢，实际上后者是对边特征的编码，前者只在multi hop中用到，是融合不同head中特征的，完全可以用线性层代替（注意如果用线性层是使用了128个不共享参数的）或`nn.Parameter()`。实际上是嵌入后的边特征与reshape后的`edge_dis_encoder`做矩阵乘法（size[max_multi_hop, num_graph\*num_node\*num_node, head]\*size[max_multi_hop, head, head]），最后的输出维度是[max_multi_hop, num_graph\*num_node\*num_node, head]。

### 3、任务的输出层：
```python
self.downstream_out_proj = nn.Linear(
	hidden_dim, get_dataset(dataset_name)['num_class'])
	
output = self.downstream_out_proj(output[:, 0, :])  # virtual node
```
这里的输出层就比较简单了只需要提取虚拟节点的所有输出特征再根据任务做一个单层的线性映射，输出分类维度或回归维度。我们注意到对‘ogbg-molhiv’数据集的输出有特殊的处理，这里我们不研究ogbg-molhiv数据集因此不对此深入分析。

下一章笔记更新grapgmer的flag融合方法，一种增加扰动的方法，等学完更新。